# GETTIN
## Planejamento de Equipamentos Escolares
## Método: Uncapacitaded Facility Location
### Versão: 0.1
#### Fillipe O Feitosa <fillipefeitosa@ua.pt>

---


#### Modeling and Creating Decision Variables

In [1]:
# Import Libraries
from gurobi import *
import math

def distance(a,b):
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    return math.sqrt(dx*dx + dy*dy)

# Data
# Problem data
subSections = [[0, 1.5],[2.5, 1.2]]
schools = [[0,0],[0,1],[0,1],
              [1,0],[1,1],[1,2],
              [2,0],[2,1],[2,2]]
charge = [3,2,3,1,3,3,4,3,2]

numSchools = len(schools)
numSubSections = len (subSections)

# Creting Guroby Model
m = Model()

# Decision Variables
x = {}
y = {}
d = {} # Distance Matrix
alpha = 1

# creating binary variable for every school
for j in range(numSchools):
    x[j] = m.addVar(vtype=GRB.BINARY, name="x%d" % j)

# creating continuous variable for subsections to check suply fractions
for i in range(numSubSections):
    for j in range(numSchools):
        # Fractions of Subsection Suply
        y[(i,j)] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="t%d,%d" % (i,j))
        # Distances between Subsections and Schools
        d[(i,j)] = distance(subSections[i], schools[j])

m.update()

#### Adding Constraints

In [2]:
# Constraint for Every Student on School
for i in range(numSubSections):
    m.addConstr(quicksum(y[(i,j)] for j in range(numSchools)) == 1)

for i in range(numSubSections):
    for j in range(numSchools):
        m.addConstr(y[(i,j)] <= x[j])

#### Objetive


In [3]:
# Setting objective

m.setObjective( quicksum(charge[j]*x[j] + quicksum(alpha*d[(i,j)]*y[(i,j)]
                for i in range(numSubSections)) for j in range(numSchools)) )


m.optimize()

Optimize a model with 20 rows, 27 columns and 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 20 rows, 27 columns, 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)

Root relaxation: objective 4.723713e+00, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.7237129    4.72371  0.00%     -    0s

Explored 0 nodes (15 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 4.72371 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.723712908962e+00, best bound 4.723712908962e+00, gap 0.0000%


In [4]:
m

<gurobi.Model MIP instance Unnamed: 20 constrs, 27 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>